# Linear Mixed Effects analysis
## N170 Component (2 ROIs)

---
Copyright 2022 [Aaron J Newman](https://github.com/aaronjnewman), [NeuroCognitive Imaging Lab](http://ncil.science), [Dalhousie University](https://dal.ca)

Released under the [BSD 3-clause license](https://opensource.org/licenses/BSD-3-Clause)

---

## Load necessary libraries

In [ ]:
date()

In [ ]:
library(tidyverse)
library(arrow)
library(ggplot2)
library(mgcv)
library(parallel)
library(emmeans)
library(sjPlot)
library(caret)
library(stargazer)
library(yaml)

## Set parameters for this analysis

In [ ]:
component <- 'n170'
subj_prefix <- 'sub-'
cond_levels <- c('FalseFont', 'ConsonantString', 'PseudoWord', 'NovelWord', 'RealWord')

cfg <- read_yaml('../../config.yml')
component_rois <- cfg$components$n170$rois
# num parallel processes for bam(). Should be less than the number of threads supported by your CPU
num_cores <- cfg$preprocessing_settings$n_jobs

# subjects to exclude
exclude <- cfg$exclude

options("scipen"=100, "digits"=5, width=300)  # force decimal instead of scientific notation

## File paths

In [ ]:
bids_root = '../..'
erp_path <- file.path(bids_root, 'derivatives', 'erp_measurement', component, 'data')
behav_path <- file.path(bids_root, 'derivatives', 'behavioral_demographic', 'data')
raw_path <- file.path(bids_root, 'rawdata')
out_path <- file.path(bids_root, 'derivatives', 'erp_lme')

# create output directories and any higher-level directories they are nested in, if they don't exist
tab_path <- file.path(out_path, component, 'tables')
dir.create(file.path(tab_path), recursive = TRUE, showWarnings = FALSE)

fig_path <- file.path(out_path, component, 'figures')
dir.create(file.path(fig_path), recursive = TRUE, showWarnings = FALSE)

## Set options for plots

In [ ]:
set_theme(base = theme_sjplot(),
          title.size = 1.5,
          title.align = 'center',
          axis.title.size = 1.5,
          axis.textsize.x=1.1,
          axis.textsize.y=1.1,
          legend.title.size = 1.5,
          legend.item.size = 1.25,
          legend.item.backcol = 'white',
          legend.size = 1.5,
          panel.gridcol.x='white',
          panel.minor.gridcol='white'
         )

# Match colours with seaborn's colorblind palette
# in python: [matplotlib.colors.to_hex(val) for val in colors.values()]
sns_colors = c('#0173b2', '#de8f05', '#029e73', '#d55e00', '#cc78bc')             

---
## Load ERP Data

In [ ]:
files <- Sys.glob(file.path(erp_path,  paste(subj_prefix, '*', sep=''), paste('*_indiv_trials_trimmed_*.feather', sep='')))

df <- files %>% map_dfr(read_feather) %>%  
        mutate_if(is.character,as.factor) %>%
        select(-c('Component', '__index_level_0__')) %>% 
        filter(ROI %in% component_rois) %>% droplevels()
        # keep only ROIs of interest for this component


# set order of conditions
df$Condition <- with(df, factor(Condition,  levels = cond_levels))

# keep only ROIs of interest for this component
# summary(df)

## Read behavioural data from LDT

In [ ]:
acc <- read.csv(file.path(behav_path, 'ldt_acc_by_subj_wide.tsv'), sep='\t')
acc$participant_id <- as.factor(acc$participant_id)
acc %>% rename(acc_CNST = ConsonantString,
              acc_FFNT = FalseFont,
              acc_PSWD= PseudoWord,
              acc_NVWD = NovelWord,
              acc_RLWD = RealWord
              ) -> acc
# summary(acc)

In [ ]:
rt <- read.csv(file.path(behav_path, 'ldt_rt_trimmed_by_subj_wide.tsv'), sep='\t')
rt$participant_id <- as.factor(rt$participant_id)
rt %>% rename(rt_CNST = ConsonantString,
              rt_FFNT = FalseFont,
              acc_PSWD= PseudoWord,
              rt_NVWD = NovelWord,
              rt_RLWD = RealWord
              ) -> rt

# summary(rt)

### merge LDT

In [ ]:
ldt <- merge(acc, rt, by=c('participant_id'))
# summary(ldt)
df <- merge(df, ldt, by=c('participant_id'))

## Read demographic data

In [ ]:
demog <- read.csv(file.path(raw_path, 'participants.tsv'), sep='\t',
                col.names=c('participant_id', 'group', 'sex', 'hand', 'age'),
                ) %>% mutate_if(is.character,as.factor)  %>% select(-c('group'))
# summary(demog)

df <- merge(df, demog, by=c('participant_id'))

## Read d prime data and merge
calculated in `scripts/behavioral_demographic/5_LDT_stats.ipynb`

In [ ]:
dprime <- read.csv(file.path(behav_path, 'ldt_sdt_data.tsv'), sep='\t',
                   colClasses=c('factor', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric'))

df <- merge(df, dprime, by='participant_id')

## Read standardized test data

In [ ]:
behav <- read.csv(file.path(raw_path, 'participants_std_tests.tsv'), 
                sep='\t') %>% mutate_if(is.character,as.factor)

## Merge ERP and behavioural data

In [ ]:
dat <- merge(df, behav, by='participant_id') %>%
            filter(!participant_id %in% exclude)  %>% droplevels()
summary(dat)

### Visualize distributions

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)

ggplot(dat, aes(y=Amplitude, x=Condition)) + 
    geom_boxplot(aes(color=Condition)) 

## Run initial full model and visualize diagnostics

In [ ]:
dat$Condition <- fct_relevel(dat$Condition, 'FalseFont')


In [ ]:
# start timer
start_time <- Sys.time()

cl <- makeCluster(num_cores)  

mod_full <- bam(Amplitude ~ Condition * ROI
                  + s(participant_id, bs="re")
                  + s(Channel, participant_id, bs="re") 
                  + s(Condition, participant_id, bs="re"),
                  data=dat, 
                  samfrac=0.1, cluster=cl, gc.level=2)

stopCluster(cl)

# print elpased time
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
gam.check(mod_full)

## GAM Modelling
### Compute the family of models we will consider, including primary variables

In [ ]:
# start timer
start_time <- Sys.time()
cl <- makeCluster(num_cores)


cr2 <-  bam(Amplitude ~ (Condition + ROI)^2 
         + s(participant_id, bs="re"),
          data=dat, 
          method='ML',
          samfrac=0.1, cluster=cl, gc.level=2)

cr1 <-  bam(Amplitude ~ (Condition + ROI) 
         + s(participant_id, bs="re"),
          data=dat, 
          method='ML',
          samfrac=0.1, cluster=cl, gc.level=2)

c1 <-  bam(Amplitude ~ Condition 
         + s(participant_id, bs="re"),
          data=dat, 
          method='ML',
          samfrac=0.1, cluster=cl, gc.level=2)

r1 <-  bam(Amplitude ~ ROI 
         + s(participant_id, bs="re"),
          data=dat, 
          method='ML',
          samfrac=0.1, cluster=cl, gc.level=2)

stopCluster(cl)
# print elpased time
end_time <- Sys.time()
print(end_time - start_time)

### Compare family of models based on AIC
Procedure based on Wagenmakers & Farrell, 2004. Models are sorted with teh best at the top of the table.  `AIC_ratio` reflects the relative likelihood of the best model relative to another model. That is, if `AIC_ratio = 2.1` for the next-best model, then the best model is 2.1x more likely than the next-best model.

In [ ]:
AICtab <- as.data.frame(AIC(cr2, cr1, c1, r1)) 
AICtab$deltaAIC <-AICtab$AIC - min(AICtab$AIC)
AICtab$AIC_L <- round(exp(-0.5 * AICtab$deltaAIC), 4)
AICtab$wAIC <- AICtab$AIC_L / sum(AICtab$AIC_L )
AICtab$AIC_ratio <-  max(AICtab$wAIC) / AICtab$wAIC
AICtab[order(AICtab$wAIC, decreasing=TRUE),]

## Compare Random Effects structures

In [ ]:
# start timer
start_time <- Sys.time()
cl <- makeCluster(num_cores)

f2_r_escs <- bam(Amplitude ~ (Condition + ROI)^2
                  + s(Channel, participant_id, bs="re") 
                  + s(Condition, participant_id, bs="re") 
                  + s(participant_id, bs="re"),
                 data=dat, 
                  method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)

f1_r_escs <- bam(Amplitude ~ (Condition + ROI)
                  + s(Channel, participant_id, bs="re") 
                  + s(Condition, participant_id, bs="re") 
                  + s(participant_id, bs="re"),
                 data=dat, 
                  method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)


f2_r_es <- bam(Amplitude ~ (Condition + ROI)^2
                  + s(Channel, participant_id, bs="re") 
                  + s(participant_id, bs="re"),
                 data=dat, 
                  method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)

f1_r_es <- bam(Amplitude ~ (Condition + ROI)
                  + s(Channel, participant_id, bs="re") 
                  + s(participant_id, bs="re"),
                 data=dat, 
                  method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)


c1w_es <-  bam(Amplitude ~ Condition 
                 + s(Channel, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                  method='ML',
                  samfrac=0.1, cluster=cl, gc.level=2)

c1w_cs <-  bam(Amplitude ~ Condition 
                 + s(Condition, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                   method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)

c1w_escs <-  bam(Amplitude ~ Condition 
                 + s(Channel, participant_id, bs="re") 
                 + s(Condition, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                  method='ML',
                  samfrac=0.1, cluster=cl, gc.level=2)

stopCluster(cl)
# print elpased time
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
AICtab <- as.data.frame(AIC(cr2, cr1, c1, r1,
                            f2_r_escs, f2_r_es, 
                            f1_r_escs, f1_r_es, 
                            c1w_es, c1w_cs, c1w_escs
                           )) 
AICtab$deltaAIC <-AICtab$AIC - min(AICtab$AIC)
AICtab$AIC_L <- round(exp(-0.5 * AICtab$deltaAIC), 4)
AICtab$wAIC <- AICtab$AIC_L / sum(AICtab$AIC_L )
AICtab$AIC_ratio <-  max(AICtab$wAIC) / AICtab$wAIC
AICtab[order(AICtab$AIC, decreasing=FALSE),]

In [ ]:
max(AICtab$wAIC)

---
## View results of best model

The full model we fit at the start was confired as best via AIC comparisons

In [ ]:
mod <- mod_full

### Plot Marginal Means of Condition x ROI


In [ ]:
terms <- c('Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=16, repr.plot.height=9)
set_theme(base = theme_sjplot(),
          title.size = 1.5,
          title.align = 'center',
          axis.title.size = 1.5,
          axis.textsize.x=1.1,
          axis.textsize.y=1.1,
          legend.title.size = 1.5,
          legend.item.size = 1.25,
          legend.item.backcol = 'white',
          legend.size = 1.5,
          panel.gridcol.x='white',
          panel.minor.gridcol='white'
         )
pm <- function(mod, terms) {
                            plot_model(mod, type='emm', terms=terms,
                                title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
                                dot.size=4, line.size=1.5, dodge=.33,
                                    ) + theme(aspect.ratio=9/16, 
                                            axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
                                            )
}
pdf(out_stem)
pm(mod_full, terms)
dev.off() 
pm(mod_full, terms)


### Laterality Effects for Each Condition

In [ ]:
terms <- c('Condition', 'ROI')
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '_LvsR', sep='')

Contrasts.emm <- emmeans(mod_full, pairwise ~ ROI | Condition)
contr_tab <- as.data.frame(Contrasts.emm$contrasts)
# contr_tab

eff_tab <- eff_size(Contrasts.emm,
                     sigma = sigma(mod_full), 
                     edf = contr_tab$df[1])
eff_tab <- as.data.frame(eff_tab)
# eff_tab

full_tab <- contr_tab %>% full_join(select(eff_tab, 'Condition', 'effect.size',  'lower.CL', 'upper.CL'),
                                    by=c('Condition'), 
                                    suffix = c(".ctr", ".eff"))  %>% 
                            mutate(across(c('p.value', 'effect.size'), round, 3)) %>% 
                            mutate(across(c('estimate', 'SE', 't.ratio'), round, 2)) %>% 
                            mutate(across(c('lower.CL', 'upper.CL'), round, 3)) %>% 
                            rename('Estimate (µV)' = 'estimate',
                                    't' = 't.ratio',
                                    'p' = 'p.value',
                                    'Effect Size' = 'effect.size',
                                    'CL low' = 'lower.CL',
                                    'CL high' = 'upper.CL'
                            ) %>% 
                            select('Condition', 'contrast', 'Estimate (µV)', 'SE', 
                            'Effect Size', 'CL low', 'CL high', 't', 'p')  %>% 
                            mutate(Condition = factor(Condition, levels = cond_levels)) %>% 
                            arrange(Condition)

full_tab

# Write table to CSV
# write.csv(full_tab, 
#           file=paste(out_stem, '.csv', sep=''),
#           row.names=FALSE
#          )

## Write table to LaTeX
tmp <- capture.output(stargazer(full_tab, 
                                summary=FALSE, rownames=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), '_LvsR', sep=''),
                                title='Between-hemisphere contrasts for each condition, from the linear mixed effects analysis. All p values are corrected for multiple comparisons using Tukey\'s method'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

### Contrasts by ROI

In [ ]:
terms <- c('Condition', 'ROI')
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '_contrasts', sep='')

Contrasts.emm <- emmeans(mod_full, pairwise ~ Condition | ROI)
contr_tab <- as.data.frame(Contrasts.emm$contrasts) 

eff_tab <- eff_size(Contrasts.emm,
                     sigma = sigma(mod_full), 
                     edf = contr_tab$df[1])
eff_tab <- as.data.frame(eff_tab) 
# replace double-bracket Conditions with single-bracket to match above
eff_tab$contrast <- contr_tab$contrast

full_tab <- contr_tab %>% full_join(select(eff_tab, 'ROI', 'contrast', 'effect.size',  'lower.CL', 'upper.CL'),
                                    by=c('contrast', 'ROI'), 
                                    suffix = c(".ctr", ".eff"))  

full_tab <- full_tab %>% 
                            mutate(across(c('p.value', 'effect.size'), round, 3)) %>% 
                            mutate(across(c('estimate', 'SE', 't.ratio'), round, 2)) %>% 
                            mutate(across(c('lower.CL', 'upper.CL'), round, 3)) %>% 
                            rename('Contrast' = 'contrast',
                                    'Estimate (µV)' = 'estimate',
                                    't' = 't.ratio',
                                    'p' = 'p.value',
                                    'Effect Size' = 'effect.size',
                                    'CL low' = 'lower.CL',
                                    'CL high' = 'upper.CL'
                            ) %>% 
                            select('Contrast', 'ROI', 'Estimate (µV)', 'SE', 
                            'Effect Size', 'CL low', 'CL high', 't', 'p')  
                            
full_tab 

# Write table to CSV
# write.csv(full_tab, 
#           file=paste(out_stem, '.csv', sep='')
#          )

## Write table to LaTeX
tmp <- capture.output(stargazer(full_tab, 
                                summary=FALSE, rownames=FALSE, font.size="footnotesize",digits=NA, initial.zero=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), '_contrasts', sep=''),
                                title='Pairwise between-condition contrasts for the N170 component, within each ROI, from the linear mixed effects analysis. All $p$ values are corrected for multiple comparisons using Tukey\'s method. Effect size is the standardized mean difference.'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

In [ ]:
options(repr.plot.width=6, repr.plot.height=12)
plot(n170.emm.cr, comparisons = TRUE, horizontal=FALSE)

In [ ]:
n170.cells <- emmeans(mod, ~ Condition * ROI)
options(repr.plot.width=15, repr.plot.height=6)
set_theme(panel.gridcol.x='white')
pwpp(n170.cells, by = "ROI", type = "response")

### Condition X ROI interactions

In [ ]:
terms <- c('Condition', 'ROI')
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '_interaction', sep='')

intxn.emm <- contrast(Contrasts.emm[[1]], interaction = c("pairwise"), by = NULL)
# intxn.emm <- as.data.frame(intxn.emm)
intxn.emm <- intxn.emm %>% as.data.frame() %>% 
                            mutate(across(c('p.value'), round, 3)) %>% 
                            mutate(across(c('estimate', 'SE', 't.ratio'), round, 2)) %>% 
                            rename('Contrast' = 'Condition_pairwise',
                                    'ROI' = 'ROI_pairwise',
                                    'Estimate (µV)' = 'estimate',
                                    't' = 't.ratio',
                                    'p' = 'p.value',
                            ) %>% 
                            select('Contrast', 'ROI', 'Estimate (µV)', 'SE', 't', 'p')  
                            
intxn.emm 


tmp <- capture.output(stargazer(intxn.emm , 
                                summary=FALSE, rownames=FALSE, font.size="footnotesize",digits=NA, initial.zero=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), '_intxn', sep=''),
                                title='Condition $\times$ ROI interaction (i.e., laterality contrast for each pairwise condition contrast) for the N170 component from the linear mixed effects analysis. All $p$ values are corrected for multiple comparisons using Tukey\'s method. Effect size is the standardized mean difference.'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

---
## Behavioural Correlations

### Compare all behavioral correlation models 
Here, `AIC_ratio` is relative to the "base" model that did not include a behavioral variable 

In [ ]:
# Refit all models using max liklihood so they can be compared:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_ortho_choice <- bam(Amplitude ~ (Condition * ROI * Ortho_Choice)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

mod_ortho_know <- bam(Amplitude ~ (Condition * ROI * Ortho_Know)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

mod_OrthoSem_Know <- bam(Amplitude ~ (Condition * ROI * OrthoSem_Know)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)       
            
mod_Sem_Choice <- bam(Amplitude ~ (Condition * ROI * Sem_Choice)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_acc_NVWD <- bam(Amplitude ~ (Condition * ROI * acc_NVWD)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_rt_NVWD <- bam(Amplitude ~ (Condition * ROI * rt_NVWD)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)             

mod_TOWRE <- bam(Amplitude ~ (Condition * ROI * TOWRE_idx)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_PsgComp <- bam(Amplitude ~ (Condition * ROI * PsgComp_percentile)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)     

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
ref_mod <- 'f2_r_escs'
AICtab <- as.data.frame(AIC( f2_r_escs,
                        mod_ortho_choice, mod_ortho_know, 
                        mod_OrthoSem_Know, mod_Sem_Choice, 
                        mod_acc_NVWD, mod_rt_NVWD, 
                        mod_TOWRE, mod_PsgComp
                           ))
# AICtab$model = row.names(AICtab)
AICtab$deltaAIC <-AICtab$AIC - AICtab[ref_mod, ]$AIC
AICtab$AIC_L <- round(exp(-0.5 * AICtab$deltaAIC), 4)
AICtab$wAIC <- AICtab$AIC_L / sum(AICtab$AIC_L )
AICtab$AIC_ratio <-  1 / (AICtab[ref_mod, ]$wAIC / AICtab$wAIC)
AICtab[order(AICtab$AIC, decreasing=FALSE),] 

---
### Orthographic choice

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_ortho_choice <- bam(Amplitude ~ (Condition * ROI * Ortho_Choice)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_ortho_choice

In [ ]:
terms <- c('Ortho_Choice', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')


pdf(out_stem, 17, 8.5)
plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) 
dev.off() 

options(repr.plot.width=20) #, repr.plot.height=6)
plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) 

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='Ortho_Choice'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))

contr_tab <- contr_tab %>%
            select(-c('df')) %>%
            rename('Orthographic Choice' = 'Ortho_Choice.trend',
                   't' = 't.ratio',
                  ) %>% 
            arrange(Condition, ROI)
contr_tab 
## Write table to LaTeX
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), sep='')
tmp <- capture.output(stargazer(contr_tab, 
                                summary=FALSE, rownames=FALSE, font.size="footnotesize",digits=4,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), sep=''),
                                title='Results of linear mixed effects model including accuracy on the orthographic choice task (indicating learning of novel word spellings in the OSLT). Statistical values are for the test of significance of the slope of the relationship between N170 amplitude and novel word accuracy'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

---
### Semantic choice

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_Sem_Choice <- bam(Amplitude ~ (Condition * ROI * Sem_Choice)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_Sem_Choice

In [ ]:
terms <- c('Sem_Choice', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='Sem_Choice'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

# eff_tab <- as.data.frame(emtrends(mod, ~ Contrast * ROI, var='TOWRE_idx'))
# eff_tab

#### Merge
# full_tab <- contr_tab %>% full_join(select(eff_tab, 'Contrast', 'ROI', 'lower.CL', 'upper.CL'), 
#                                     by=c('Contrast', 'ROI'), 
#                                     suffix = c(".ctr", ".eff"))
# full_tab


---
### OrthoSem_Know

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_OrthoSem_Know <- bam(Amplitude ~ (Condition * ROI * OrthoSem_Know)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_OrthoSem_Know

In [ ]:
terms <- c('OrthoSem_Know', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='OrthoSem_Know'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Novel Word Accuracy

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)
mod_acc_NVWD <- bam(Amplitude ~ (Condition * ROI * acc_NVWD)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
terms <- c('acc_NVWD', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)
# set_theme(panel.gridcol.x='white'))
pm <- function(mod, terms){
    plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) 
}

# save to PDF
pdf(out_stem, 20, 8.5)
pm(mod_acc_NVWD, terms)
dev.off() 
# show in notebook
pm(mod_acc_NVWD, terms)

In [ ]:
terms <- c('acc_NVWD', 'Condition', 'ROI')

contr_tab <- test(emtrends(mod_acc_NVWD, ~ Condition * ROI, var='acc_NVWD'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value, df))

contr_tab <- contr_tab %>%
            rename('Novel Word Accuracy' = 'acc_NVWD.trend',
                #    'SE' = 'SE.ctr',
                   't' = 't.ratio',
                #    'Effect Size' = 'effect.size'
                  ) %>% 
            arrange(Condition, ROI)
contr_tab 
# Write table to LaTeX
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), sep='')
tmp <- capture.output(stargazer(contr_tab, 
                                summary=FALSE, rownames=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), sep=''),
                                title='Results of linear mixed effects model including accuracy on novel words in the lexical decision task. Statistical values are for the test of significance of the slope of the relationship between N170 amplitude and novel word accuracy'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

#### interaction

In [ ]:
trend_tab <- emtrends(mod_acc_NVWD, pairwise ~ Condition , var='acc_NVWD')
trend_tab$contrasts

In [ ]:
trend_tab <- emtrends(mod_acc_NVWD, pairwise ~ ROI | Condition, var='acc_NVWD')
trend_tab$contrasts

In [ ]:
trend_tab <- emtrends(mod_acc_NVWD, pairwise ~ Condition | ROI, var='acc_NVWD')
trend_tab$contrasts

---
### Novel word RT

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_rt_NVWD <- bam(Amplitude ~ (Condition * ROI * rt_NVWD)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_rt_NVWD

In [ ]:
terms <- c('rt_NVWD', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='rt_NVWD'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

# eff_tab <- as.data.frame(emtrends(mod, ~ Contrast * ROI, var='TOWRE_idx'))
# eff_tab

#### Merge
# full_tab <- contr_tab %>% full_join(select(eff_tab, 'Contrast', 'ROI', 'lower.CL', 'upper.CL'), 
#                                     by=c('Contrast', 'ROI'), 
#                                     suffix = c(".ctr", ".eff"))
# full_tab


---
### TOWRE

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_TOWRE <- bam(Amplitude ~ (Condition * ROI * TOWRE_idx)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_TOWRE

In [ ]:
terms <- c('TOWRE_idx', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='TOWRE_idx'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

# eff_tab <- as.data.frame(emtrends(mod, ~ Contrast * ROI, var='TOWRE_idx'))
# eff_tab

#### Merge
# full_tab <- contr_tab %>% full_join(select(eff_tab, 'Contrast', 'ROI', 'lower.CL', 'upper.CL'), 
#                                     by=c('Contrast', 'ROI'), 
#                                     suffix = c(".ctr", ".eff"))
# full_tab


---
## Reading comprehension

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_PsgComp <- bam(Amplitude ~ (Condition * ROI * PsgComp_percentile)
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            # method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
terms <- c('PsgComp_percentile', 'Condition', 'ROI')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=20) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,   colors=sns_colors
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition * ROI, var='PsgComp_percentile'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

# eff_tab <- as.data.frame(emtrends(mod, ~ Contrast * ROI, var='TOWRE_idx'))
# eff_tab

#### Merge
# full_tab <- contr_tab %>% full_join(select(eff_tab, 'Contrast', 'ROI', 'lower.CL', 'upper.CL'), 
#                                     by=c('Contrast', 'ROI'), 
#                                     suffix = c(".ctr", ".eff"))
# full_tab
